<a href="https://colab.research.google.com/github/victordotm/TFM/blob/master/Crear_pickle_database.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
BASE_FOLDER = "/content/drive/My Drive/Colab Notebooks/TFM/"
BASE_FOLDER_test = BASE_FOLDER+"Dataset_test/"
BASE_FOLDER_VGG16 = BASE_FOLDER+"Dataset_filtroVGG16_Imagenet-reviewed/"
BASE_FOLDER_YOLOv3 = BASE_FOLDER+"Dataset_filtroYOLOv3-reviewed/"
BASE_FOLDER_manual = BASE_FOLDER+"Dataset_manual_filter/"
BASE_FOLDER_sinFiltrar = BASE_FOLDER+"Dataset_noDup-reviewed/"
BASE_FOLDER_testFinal = BASE_FOLDER+"Dataset_TestFinal/"

from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [0]:
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
import random
import pickle
import time
from scipy import ndarray
from tqdm import tqdm
import skimage as sk
from skimage import transform
from skimage import util

## Definición funciones & utiles

#### Decorador: util para calcular el tiempo de ejecución de las funciones

In [0]:
#Decorador para calcular_tiempo a la función para la cual queremos calcular el tiempo de ejecución


#Función para calcular el tiempo de ejecución
def calcular_tiempo(f): # @calcular_tiempo
   
    def wrapper(*args, **kwargs):        
        inicio = time.time()       
        resultado = f(*args, **kwargs)       
        tiempo = time.time() - inicio
        print("Tiempo de ejecución para algoritmo: "+str(tiempo))
        return resultado
    
    return wrapper

#### Definición funciones de transformanción de imagenes

In [0]:
def random_rotation(image_array: ndarray, rot_left=-25, rot_right=25):
    #Nos aseguramos que cada llamada a la función generará una random diferente
    random_param = time.clock()
    random_param = random_param*rot_left
    random.seed(random_param)
    
    # tomamos un numero random de grados a rotar dentro del intervalo
    random_degree = random.uniform(rot_left, rot_right)
    return sk.transform.rotate(image_array, random_degree)

def random_noise(image_array: ndarray, seed_random = 1):
  
    #Nos aseguramos que cada llamada a la función generará una random diferente
    random_param = time.clock()
    random_param = random_param*seed_random
    random.seed(random_param)
    
    # add random noise to the image
    return sk.util.random_noise(image_array)

def horizontal_flip(image_array: ndarray):
    # Inversión del array de pixeles
    return image_array[:, ::-1]

#### Definición funciones de data augmentation

In [0]:
def Aug_data_4 (training_data, new_array, class_num):
  
  training_data.append([new_array, class_num]) #guardamos imagen + label originales
  
  #aplicamos tecnicas de data augmentation
  
  #rotación
  array_rotated=random_rotation(new_array)
  training_data.append([array_rotated, class_num]) #guardamos imagen + label
  
  #ruido
  array_ruido=random_noise(new_array)
  training_data.append([array_ruido, class_num]) #guardamos imagen + label
  
  #giro horizontal
  array_giro=horizontal_flip(new_array)
  training_data.append([array_giro, class_num]) #guardamos imagen + label
  
  return training_data

In [0]:

def Aug_data_6 (training_data, new_array, class_num):

  training_data.append([new_array, class_num]) #guardamos imagen + label originales
  #aplicamos tecnicas de data augmentation
  Aug_array = []
  
  #rotación
  array_rotated=random_rotation(new_array)
  Aug_array.append(new_array)
  Aug_array.append(array_rotated) #guardamos imagen + label
  
  for element in range(len(Aug_array)):
    #ruido
    array_ruido=random_noise(Aug_array[element])
    Aug_array.append(array_ruido) #guardamos imagen + label

  for element in range(len(Aug_array)): 
    #giro horizontal
    array_giro=horizontal_flip(Aug_array[element])
    Aug_array.append(array_giro) #guardamos imagen + label
    
  for element in range(len(Aug_array)): 
    training_data.append([Aug_array[element], class_num])

  return training_data

In [0]:
def Aug_data_max (training_data, new_array, class_num):

  training_data.append([new_array, class_num]) #guardamos imagen + label originales
  #aplicamos tecnicas de data augmentation
  Aug_array = []
  Aug_array.append(new_array) #guardamos la imagen inicial
  
  #rotación
  for it in range(2,5): #para el caso de filtro manual: (3,5)
    rotations = it/1.2
    r_left=10
    r_right=10
    array_rotated=random_rotation(new_array, r_left*rotations, r_right*rotations)
    Aug_array.append(array_rotated) #guardamos imagen + label
  
  for element in range(len(Aug_array)):
    #ruido
    array_ruido=random_noise(Aug_array[element],element+1)
    Aug_array.append(array_ruido) #guardamos imagen + label

  for element in range(len(Aug_array)): 
    #giro horizontal
    array_giro=horizontal_flip(Aug_array[element])
    Aug_array.append(array_giro) #guardamos imagen + label
    
  for element in range(len(Aug_array)): 
    training_data.append([Aug_array[element], class_num])

  return training_data

In [0]:
def ratio_dataAug(num_inicial, out_array):
  num_final = len(out_array)
  
  ratio_aug = num_final / num_inicial
  
  return ratio_aug
  

#### Definición función creación datos de entrenamiento:
Data augmentation, sin balancear

In [0]:
@calcular_tiempo
def create_training_data(categories, folder, img_size, Aug_level):
  
  training_data = []
  num_images_inicial = 0
  
  for category in categories:  

    path = os.path.join(folder,category)  
    class_num = categories.index(category)  #Cada nombre de carpeta equivale a una clase
    
    for img in tqdm(os.listdir(path)):  
      img_array = cv2.imread(os.path.join(path,img))  
      new_array = cv2.resize(img_array, (img_size, img_size)) #redimencionamos images para homogenizar su tamaño
      #Llevamos la cuenta del número de images originales que hay en el database
      num_images_inicial = num_images_inicial + 1
      
      #Aplicamos diferentes niveles de dataAugmentation según argumento
      if Aug_level == 0:
        training_data.append([new_array, class_num]) #guardamos imagen + label
      elif Aug_level == 4:
        training_data = Aug_data_4 (training_data, new_array, class_num)
      elif Aug_level == 6:
        training_data = Aug_data_6 (training_data, new_array, class_num)
      elif Aug_level == 8:
        training_data = Aug_data_max (training_data, new_array, class_num)

  print("Ratio data Augmentation:")      
  print(ratio_dataAug(num_images_inicial,training_data))
  
  print("")
  print("Función ejecutada: create_training_data")
  
  return training_data

#### Definición función creación datos de entrenamiento:
Base de datos balanceadas

In [0]:
@calcular_tiempo
def balanced_training_data(categories, folder, img_size, Aug_level, target_size):
  
  training_data = []
  
  for category in categories:  

    path = os.path.join(folder,category)  
    class_num = categories.index(category)  #Cada nombre de carpeta equivale a una clase
    num_images = 0 # nueva categoria, reiniciamos contador número imagenes
    
    category_data = []
    
    for img in tqdm(os.listdir(path)):
      
      img_array = cv2.imread(os.path.join(path,img))  
      new_array = cv2.resize(img_array, (img_size, img_size)) #redimencionamos images para homogenizar su tamaño
      category_data.append(new_array)# guardamos imagen actual + label
      
      if (target_size - len(category_data)) == 0: #Si alcanzamos el número de imagenes necesarias dejamos de leer el fichero

        break
        
    loop=0 #Contador numero de loops while data Augmentation
      
    while (target_size - len(category_data)) >= 0:

      loop = loop + 1
      rot_left = (-25-loop)
      rot_right = (25+loop)
        
      for element in range(len(category_data)):
        #rotación
        array_rotated=random_rotation(category_data[element], rot_left, rot_right)
        category_data.append(array_rotated) #guardamos imagen + label
  
        #ruido
        array_ruido=random_noise(category_data[element], loop*rot_right)
        category_data.append(array_ruido) #guardamos imagen + label
 
        if loop == 1:
          #giro horizontal
          #Solo podemos aplicar el giro horizontal a la imagen original 1 vez
          array_giro=horizontal_flip(category_data[element])
          category_data.append(array_giro) #guardamos imagen + label
          
        if (target_size - len(category_data)) <= 0: #Si alcanzamos el número de imagenes necesarias dejamos de crear imagenes
          break
            
      if (target_size - len(category_data)) <= 0: #Si alcanzamos el número de imagenes necesarias dejamos de crear imagenes
        break

    for element in range(len(category_data)): 
      training_data.append([category_data[element], class_num])
    
    print("")
    print("")
    print("Imagenes Categoria:",class_num)
    print(len(category_data))
    print("")
  
  print("")
  print("Función ejecutada: create_training_data")
  
  return training_data

#### Definición creación base de datos en formato pickle

In [0]:
@calcular_tiempo
def create_pickle_database(BASE_FOLDER_data, categories, img_size, pick_name, Aug_level, target_size = 0):
  
  if target_size == 0:
    training_data = create_training_data(categories, BASE_FOLDER_data, img_size, Aug_level)
  
  else:
    training_data = balanced_training_data(categories, BASE_FOLDER_data, img_size, Aug_level, target_size)
  
  print("Tamaño base de datos pickle:")
  print(len(training_data))
  
  #Mezclamos de forma random los datos para que después sea optimo el entrenamiento
  random.shuffle(training_data)

  X = []
  y = []

  for features,label in training_data:
    X.append(features)
    y.append(label)

  X = np.array(X).reshape(-1, img_size, img_size, 3)

  
  pickle_out = open(BASE_FOLDER_data+"Imag"+pick_name+".pickle","wb")
  pickle.dump(X, pickle_out)
  pickle_out.close()

  pickle_out = open(BASE_FOLDER_data+"Label"+pick_name+".pickle","wb")
  pickle.dump(y, pickle_out)
  pickle_out.close()
  
  print("Función ejecutada: create_pickle_database")
  
  return True

#### Definición variables 

In [0]:
#definimos categorias
Complete_cat = ["acciona", "blinkee","ecooltra","muving","motos","yego"]
Binary_cat = ["motos","electrica"]

#definimos image size
img_size = 104 #definimos tamaño de imagenes

#definimos pickle name

#Categorias completas / sin balancear
pick_name_std_ub_complet = "std_ub_c"
pick_name_Aug4_ub_complet = "Aug4_ub_c"
pick_name_Aug6_ub_complet = "Aug6_ub_c"
pick_name_AugMx_ub_complet = "AugMx_ub_c"


#Categorias completas / balanceadas
pick_name_low_bal = "low_bal"
pick_name_medium_bal = "medium_bal"
pick_name_high_bal = "high_bal"

#Categorias binarias / sin balancear
pick_name_std_ub_bin = "std_ub_bin"
pick_name_Aug4_ub_bin = "Aug4_ub_bin"
pick_name_Aug6_ub_bin = "Aug6_ub_bin"
pick_name_AugMx_ub_bin = "AugMx_ub_bin"

#definimos tamaño objetivo balanceo
low_bal = 200
medium_bal = 500
high_bal = 1000

#definimos niveles de dataAugmentation a aplicar
Aug_level_0 = 0
Aug_level_4 = 4
Aug_level_6 = 6
Aug_level_max = 8

## Creación base de datos

### Crear bases de datos basadas en imagenes YOLOv3

#### Creamos base de datos sin balancear categorias

#### 1. Base de datos YOLOv3 maximo nivel data augmentation, todas las categorias

In [0]:
create_pickle_database(BASE_FOLDER_YOLOv3, Complete_cat, img_size, pick_name_AugMx_ub_complet, Aug_level_max)

100%|██████████| 24/24 [00:00<00:00, 35.94it/s]


Ratio data Augmentation:
17.0

Función ejecutada: create_training_data
Tiempo de ejecución para algoritmo: 30.910341501235962
Tamaño base de datos pickle:
15963
Función ejecutada: create_pickle_database
Tiempo de ejecución para algoritmo: 144.87870907783508


True

#### 2. Base de datos YOLOv3 nivel medio data augmentation, todas las categorias

In [0]:
create_pickle_database(BASE_FOLDER_YOLOv3, Complete_cat, img_size, pick_name_Aug6_ub_complet, Aug_level_6)

100%|██████████| 24/24 [00:00<00:00, 47.07it/s]


Ratio data Augmentation:
9.0

Función ejecutada: create_training_data
Tiempo de ejecución para algoritmo: 27.18750524520874
Tamaño base de datos pickle:
8451
Función ejecutada: create_pickle_database
Tiempo de ejecución para algoritmo: 60.19911432266235


True

#### 3. Base de datos YOLOv3 nivel bajo data augmentation, todas las categorias

In [0]:
create_pickle_database(BASE_FOLDER_YOLOv3, Complete_cat, img_size, pick_name_Aug4_ub_complet, Aug_level_4)

100%|██████████| 24/24 [00:00<00:00, 62.57it/s]


Ratio data Augmentation:
4.0

Función ejecutada: create_training_data
Tiempo de ejecución para algoritmo: 19.84935688972473
Tamaño base de datos pickle:
3756
Función ejecutada: create_pickle_database
Tiempo de ejecución para algoritmo: 26.008620738983154


True

#### 4. Base de datos YOLOv3 sin data augmentation, todas las categorias

In [0]:
create_pickle_database(BASE_FOLDER_YOLOv3, Complete_cat, img_size, pick_name_std_ub_complet, Aug_level_0)

100%|██████████| 24/24 [00:00<00:00, 85.17it/s]


Ratio data Augmentation:
1.0

Función ejecutada: create_training_data
Tiempo de ejecución para algoritmo: 15.745143413543701
Tamaño base de datos pickle:
939
Función ejecutada: create_pickle_database
Tiempo de ejecución para algoritmo: 16.08640146255493


True

#### Creamos base de datos balanceadas

#### 1. Base de datos YOLOv3 balanceada, todas las categorias, tamaño base de datos final nivel bajo

In [0]:
create_pickle_database(BASE_FOLDER_YOLOv3, Complete_cat, img_size, pick_name_low_bal, Aug_level_0, low_bal)

  1%|          | 1/109 [00:00<00:11,  9.27it/s]



Imagenes Categoria: 0
200



  7%|▋         | 11/167 [00:00<00:01, 105.94it/s]



Imagenes Categoria: 1
202



  2%|▏         | 2/133 [00:00<00:09, 13.21it/s]

...

 38%|███▊      | 9/24 [00:00<00:00, 81.62it/s]



Imagenes Categoria: 4
203



100%|██████████| 24/24 [00:00<00:00, 81.85it/s]




Imagenes Categoria: 5
200


Función ejecutada: create_training_data
Tiempo de ejecución para algoritmo: 11.18227767944336
Tamaño base de datos pickle:
1207
Función ejecutada: create_pickle_database
Tiempo de ejecución para algoritmo: 13.585546255111694


True

#### 2. Base de datos YOLOv3 balanceada, todas las categorias, tamaño base de datos final nivel medio

In [0]:
create_pickle_database(BASE_FOLDER_YOLOv3, Complete_cat, img_size, pick_name_medium_bal, Aug_level_0, medium_bal)

  1%|          | 1/109 [00:00<00:11,  9.21it/s]



Imagenes Categoria: 0
500



  6%|▌         | 10/167 [00:00<00:01, 92.53it/s]



Imagenes Categoria: 1
500



  2%|▏         | 2/133 [00:00<00:09, 13.78it/s]

...

 42%|████▏     | 10/24 [00:00<00:00, 99.38it/s]



Imagenes Categoria: 4
501



100%|██████████| 24/24 [00:00<00:00, 98.41it/s]




Imagenes Categoria: 5
500


Función ejecutada: create_training_data
Tiempo de ejecución para algoritmo: 18.629858255386353
Tamaño base de datos pickle:
3003
Función ejecutada: create_pickle_database
Tiempo de ejecución para algoritmo: 21.99893021583557


True

#### 3. Base de datos YOLOv3 balanceada, todas las categorias, tamaño base de datos final nivel alto

In [0]:
create_pickle_database(BASE_FOLDER_YOLOv3, Complete_cat, img_size, pick_name_high_bal, Aug_level_0, high_bal)

  1%|          | 1/109 [00:00<00:12,  8.71it/s]



Imagenes Categoria: 0
1000



  7%|▋         | 12/167 [00:00<00:01, 89.88it/s]



Imagenes Categoria: 1
1000



  2%|▏         | 2/133 [00:00<00:10, 12.53it/s]

...

 38%|███▊      | 9/24 [00:00<00:00, 84.57it/s]



Imagenes Categoria: 4
1002



100%|██████████| 24/24 [00:00<00:00, 88.52it/s]




Imagenes Categoria: 5
1000


Función ejecutada: create_training_data
Tiempo de ejecución para algoritmo: 23.538779735565186
Tamaño base de datos pickle:
6002
Función ejecutada: create_pickle_database
Tiempo de ejecución para algoritmo: 43.26490259170532


True

### Crear bases de datos basadas en imagenes VGG16

#### Creamos base de datos sin balancear categorias

#### 1. Base de datos VGG16 maximo nivel data augmentation, todas las categorias

In [0]:
create_pickle_database(BASE_FOLDER_VGG16 , Complete_cat, img_size, pick_name_AugMx_ub_complet, Aug_level_max)

100%|██████████| 1/1 [00:00<00:00,  1.82it/s]


Ratio data Augmentation:
17.0

Función ejecutada: create_training_data
Tiempo de ejecución para algoritmo: 222.8458001613617
Tamaño base de datos pickle:
6596
Función ejecutada: create_pickle_database
Tiempo de ejecución para algoritmo: 243.58773708343506


True

#### 2. Base de datos VGG16 nivel medio data augmentation, todas las categorias

In [0]:
create_pickle_database(BASE_FOLDER_VGG16, Complete_cat, img_size, pick_name_Aug6_ub_complet, Aug_level_6)

100%|██████████| 1/1 [00:00<00:00, 101.06it/s]


Ratio data Augmentation:
9.0

Función ejecutada: create_training_data
Tiempo de ejecución para algoritmo: 3.700449228286743
Tamaño base de datos pickle:
3492
Función ejecutada: create_pickle_database
Tiempo de ejecución para algoritmo: 7.766133785247803


True

#### 3. Base de datos VGG16 nivel bajo data augmentation, todas las categorias

In [0]:
create_pickle_database(BASE_FOLDER_VGG16, Complete_cat, img_size, pick_name_Aug4_ub_complet, Aug_level_4)

100%|██████████| 1/1 [00:00<00:00, 134.55it/s]


Ratio data Augmentation:
4.0

Función ejecutada: create_training_data
Tiempo de ejecución para algoritmo: 2.7648558616638184
Tamaño base de datos pickle:
1552
Función ejecutada: create_pickle_database
Tiempo de ejecución para algoritmo: 4.301688194274902


True

#### 4. Base de datos VGG16 sin data augmentation, todas las categorias

In [0]:
create_pickle_database(BASE_FOLDER_VGG16, Complete_cat, img_size, pick_name_std_ub_complet, Aug_level_0)

100%|██████████| 1/1 [00:00<00:00, 367.34it/s]


Ratio data Augmentation:
1.0

Función ejecutada: create_training_data
Tiempo de ejecución para algoritmo: 1.2998034954071045
Tamaño base de datos pickle:
388
Función ejecutada: create_pickle_database
Tiempo de ejecución para algoritmo: 1.3837780952453613


True

#### Creamos base de datos balanceadas

#### 1. Base de datos VGG16 balanceada, todas las categorias, tamaño base de datos final nivel bajo

In [0]:
create_pickle_database(BASE_FOLDER_VGG16, Complete_cat, img_size, pick_name_low_bal, Aug_level_0, low_bal)

100%|██████████| 12/12 [00:00<00:00, 314.42it/s]



Imagenes Categoria: 0
200




100%|██████████| 39/39 [00:00<00:00, 316.72it/s]



Imagenes Categoria: 1
200




100%|██████████| 22/22 [00:00<00:00, 356.76it/s]

...


  7%|▋         | 23/310 [00:00<00:01, 224.74it/s]



Imagenes Categoria: 3
200



100%|██████████| 1/1 [00:00<00:00, 168.13it/s]




Imagenes Categoria: 4
203



Imagenes Categoria: 5
200


Función ejecutada: create_training_data
Tiempo de ejecución para algoritmo: 2.496094226837158
Tamaño base de datos pickle:
1203
Función ejecutada: create_pickle_database
Tiempo de ejecución para algoritmo: 3.9155828952789307


True

#### 2. Base de datos VGG16 balanceada, todas las categorias, tamaño base de datos final nivel medio

In [0]:
create_pickle_database(BASE_FOLDER_VGG16, Complete_cat, img_size, pick_name_medium_bal, Aug_level_0, medium_bal)

100%|██████████| 12/12 [00:00<00:00, 388.00it/s]



Imagenes Categoria: 0
500




100%|██████████| 39/39 [00:00<00:00, 447.65it/s]



Imagenes Categoria: 1
500




100%|██████████| 22/22 [00:00<00:00, 370.93it/s]

...



Imagenes Categoria: 3
500



100%|██████████| 1/1 [00:00<00:00, 230.42it/s]



Imagenes Categoria: 4
502





Imagenes Categoria: 5
500


Función ejecutada: create_training_data
Tiempo de ejecución para algoritmo: 5.397103309631348
Tamaño base de datos pickle:
3002
Función ejecutada: create_pickle_database
Tiempo de ejecución para algoritmo: 8.25750732421875


True

#### 3. Base de datos VGG16 balanceada, todas las categorias, tamaño base de datos final nivel alto

In [0]:
create_pickle_database(BASE_FOLDER_VGG16, Complete_cat, img_size, pick_name_high_bal, Aug_level_0, high_bal)

100%|██████████| 12/12 [00:00<00:00, 277.07it/s]



Imagenes Categoria: 0
1000




100%|██████████| 39/39 [00:00<00:00, 320.44it/s]



Imagenes Categoria: 1
1000




100%|██████████| 22/22 [00:00<00:00, 308.04it/s]

...



Imagenes Categoria: 3
1000



100%|██████████| 1/1 [00:00<00:00, 172.12it/s]



Imagenes Categoria: 4
1000





Imagenes Categoria: 5
1000


Función ejecutada: create_training_data
Tiempo de ejecución para algoritmo: 9.308899164199829
Tamaño base de datos pickle:
6000
Función ejecutada: create_pickle_database
Tiempo de ejecución para algoritmo: 42.73024821281433


True

### Crear bases de datos basadas en imagenes filtradas manualmente

#### Creamos base de datos sin balancear categorias

#### 1. Base de datos manual maximo nivel data augmentation, todas las categorias

In [0]:
create_pickle_database(BASE_FOLDER_manual , Complete_cat, img_size, pick_name_AugMx_ub_complet, Aug_level_max)

100%|██████████| 29/29 [00:00<00:00, 57.70it/s]


Ratio data Augmentation:
13.0

Función ejecutada: create_training_data
Tiempo de ejecución para algoritmo: 24.72007393836975
Tamaño base de datos pickle:
13676
Función ejecutada: create_pickle_database
Tiempo de ejecución para algoritmo: 111.3948106765747


True

#### 2. Base de datos manual nivel medio data augmentation, todas las categorias

In [0]:
create_pickle_database(BASE_FOLDER_manual, Complete_cat, img_size, pick_name_Aug6_ub_complet, Aug_level_6)

100%|██████████| 29/29 [00:00<00:00, 60.41it/s]


Ratio data Augmentation:
9.0

Función ejecutada: create_training_data
Tiempo de ejecución para algoritmo: 23.284021377563477
Tamaño base de datos pickle:
9468
Función ejecutada: create_pickle_database
Tiempo de ejecución para algoritmo: 66.27956891059875


True

#### 3. Base de datos manual nivel bajo data augmentation, todas las categorias

In [0]:
create_pickle_database(BASE_FOLDER_manual, Complete_cat, img_size, pick_name_Aug4_ub_complet, Aug_level_4)

100%|██████████| 29/29 [00:00<00:00, 66.39it/s]


Ratio data Augmentation:
4.0

Función ejecutada: create_training_data
Tiempo de ejecución para algoritmo: 23.027857303619385
Tamaño base de datos pickle:
4208
Función ejecutada: create_pickle_database
Tiempo de ejecución para algoritmo: 43.90852165222168


True

#### 4. Base de datos manual sin data augmentation, todas las categorias

In [0]:
create_pickle_database(BASE_FOLDER_manual, Complete_cat, img_size, pick_name_std_ub_complet, Aug_level_0)

100%|██████████| 29/29 [00:00<00:00, 86.64it/s]


Ratio data Augmentation:
1.0

Función ejecutada: create_training_data
Tiempo de ejecución para algoritmo: 20.879185914993286
Tamaño base de datos pickle:
1052
Función ejecutada: create_pickle_database
Tiempo de ejecución para algoritmo: 21.13108539581299


True

#### Creamos base de datos balanceadas

#### 1. Base de datos manual balanceada, todas las categorias, tamaño base de datos final nivel bajo

In [0]:
create_pickle_database(BASE_FOLDER_manual, Complete_cat, img_size, pick_name_low_bal, Aug_level_0, low_bal)

  0%|          | 0/135 [00:00<?, ?it/s]



Imagenes Categoria: 0
200



  0%|          | 0/195 [00:00<?, ?it/s]



Imagenes Categoria: 1
201



  0%|          | 0/161 [00:00<?, ?it/s]

...

  0%|          | 0/29 [00:00<?, ?it/s]



Imagenes Categoria: 4
203




100%|██████████| 29/29 [00:18<00:00,  1.55it/s]




Imagenes Categoria: 5
200


Función ejecutada: create_training_data
Tiempo de ejecución para algoritmo: 504.67422795295715
Tamaño base de datos pickle:
1205
Función ejecutada: create_pickle_database
Tiempo de ejecución para algoritmo: 507.69601702690125


True

#### 2. Base de datos manual balanceada, todas las categorias, tamaño base de datos final nivel medio

In [0]:
create_pickle_database(BASE_FOLDER_manual, Complete_cat, img_size, pick_name_medium_bal, Aug_level_0, medium_bal)


100%|██████████| 62/62 [00:00<00:00, 62.75it/s]

  1%|          | 1/135 [00:00<00:15,  8.91it/s]



Imagenes Categoria: 0
500




  4%|▎         | 7/195 [00:00<00:02, 68.73it/s]



Imagenes Categoria: 1
501




100%|██████████| 195/195 [00:02<00:00, 79.14it/s]

  1%|          | 1/161 [00:00<00:16,  9.82it/s]

...


100%|██████████| 470/470 [03:09<00:00,  1.49it/s]

 28%|██▊       | 8/29 [00:00<00:00, 73.27it/s]



Imagenes Categoria: 4
500




100%|██████████| 29/29 [00:00<00:00, 83.56it/s]



Imagenes Categoria: 5
500


Función ejecutada: create_training_data
Tiempo de ejecución para algoritmo: 200.98614764213562
Tamaño base de datos pickle:
3002
Función ejecutada: create_pickle_database
Tiempo de ejecución para algoritmo: 207.91188645362854


True

#### 3. Base de datos manual balanceada, todas las categorias, tamaño base de datos final nivel alto

In [0]:
create_pickle_database(BASE_FOLDER_manual, Complete_cat, img_size, pick_name_high_bal, Aug_level_0, high_bal)


  1%|          | 1/135 [00:00<00:15,  8.55it/s]



Imagenes Categoria: 0
1000




  2%|▏         | 4/195 [00:00<00:04, 38.37it/s]



Imagenes Categoria: 1
1000




  1%|          | 1/161 [00:00<00:18,  8.68it/s]

...


 21%|██        | 6/29 [00:00<00:00, 56.30it/s]



Imagenes Categoria: 4
1001




100%|██████████| 29/29 [00:00<00:00, 73.05it/s]



Imagenes Categoria: 5
1000


Función ejecutada: create_training_data
Tiempo de ejecución para algoritmo: 29.12835955619812
Tamaño base de datos pickle:
6001
Función ejecutada: create_pickle_database
Tiempo de ejecución para algoritmo: 42.076083183288574


True

### Crear bases de datos basadas en imagenes no filtradas

#### Creamos base de datos sin balancear categorias

#### 1. Base de datos maximo nivel data augmentation, todas las categorias

Al tratarse de la base de datos sin filtrar, la cantidad de imagenes es más que considerable.<br>
No aplicamos el nivel máximo de dataAugmentation para este caso.

#### 2. Base de datos nivel medio data augmentation, todas las categorias

In [0]:
create_pickle_database(BASE_FOLDER_sinFiltrar, Complete_cat, img_size, pick_name_Aug6_ub_complet, Aug_level_6)

100%|██████████| 345/345 [03:39<00:00,  1.49it/s]


Ratio data Augmentation:
5.0

Función ejecutada: create_training_data
Tiempo de ejecución para algoritmo: 1232.27205824852
Tamaño base de datos pickle:
10900
Función ejecutada: create_pickle_database
Tiempo de ejecución para algoritmo: 1303.8706290721893


True

#### 3. Base de datos nivel bajo data augmentation, todas las categorias

In [0]:
create_pickle_database(BASE_FOLDER_sinFiltrar, Complete_cat, img_size, pick_name_Aug4_ub_complet, Aug_level_4)

100%|██████████| 345/345 [00:12<00:00, 28.58it/s]


Ratio data Augmentation:
4.0

Función ejecutada: create_training_data
Tiempo de ejecución para algoritmo: 50.70317053794861
Tamaño base de datos pickle:
8720
Función ejecutada: create_pickle_database
Tiempo de ejecución para algoritmo: 86.07863545417786


True

#### 4. Base de datos sin data augmentation, todas las categorias

In [0]:
create_pickle_database(BASE_FOLDER_sinFiltrar, Complete_cat, img_size, pick_name_std_ub_complet, Aug_level_0)

100%|██████████| 345/345 [00:04<00:00, 74.45it/s]


Ratio data Augmentation:
1.0

Función ejecutada: create_training_data
Tiempo de ejecución para algoritmo: 34.610584020614624
Tamaño base de datos pickle:
2180
Función ejecutada: create_pickle_database
Tiempo de ejecución para algoritmo: 35.17029333114624


True

#### Creamos base de datos balanceadas

#### 1. Base de datos balanceada, todas las categorias, tamaño base de datos final nivel bajo

In [0]:
create_pickle_database(BASE_FOLDER_sinFiltrar, Complete_cat, img_size, pick_name_low_bal, Aug_level_0, low_bal)

  0%|          | 1/247 [00:00<00:31,  7.91it/s]



Imagenes Categoria: 0
203



  2%|▏         | 7/384 [00:00<00:05, 64.86it/s]



Imagenes Categoria: 1
203



  0%|          | 2/460 [00:00<00:26, 17.53it/s]

...

  0%|          | 0/345 [00:00<?, ?it/s]



Imagenes Categoria: 4
203



 57%|█████▋    | 196/345 [00:05<00:03, 41.98it/s]




Imagenes Categoria: 5
203


Función ejecutada: create_training_data
Tiempo de ejecución para algoritmo: 24.47635507583618
Tamaño base de datos pickle:
1218
Función ejecutada: create_pickle_database
Tiempo de ejecución para algoritmo: 28.32234764099121


True

#### 2. Base de datos balanceada, todas las categorias, tamaño base de datos final nivel medio

In [0]:
create_pickle_database(BASE_FOLDER_sinFiltrar, Complete_cat, img_size, pick_name_medium_bal, Aug_level_0, medium_bal)

  0%|          | 1/247 [00:00<00:30,  8.06it/s]



Imagenes Categoria: 0
502



  2%|▏         | 7/384 [00:00<00:06, 62.66it/s]



Imagenes Categoria: 1
502



  0%|          | 2/460 [00:00<00:27, 16.53it/s]

...

  0%|          | 0/345 [00:00<?, ?it/s]



Imagenes Categoria: 4
500



100%|██████████| 345/345 [00:05<00:00, 65.47it/s]




Imagenes Categoria: 5
501


Función ejecutada: create_training_data
Tiempo de ejecución para algoritmo: 36.62601923942566
Tamaño base de datos pickle:
3008
Función ejecutada: create_pickle_database
Tiempo de ejecución para algoritmo: 40.587419509887695


True

#### 3. Base de datos balanceada, todas las categorias, tamaño base de datos final nivel alto

In [0]:
create_pickle_database(BASE_FOLDER_sinFiltrar, Complete_cat, img_size, pick_name_high_bal, Aug_level_0, high_bal)

  0%|          | 1/247 [00:00<00:28,  8.58it/s]



Imagenes Categoria: 0
1000



  2%|▏         | 6/384 [00:00<00:06, 58.28it/s]



Imagenes Categoria: 1
1000



  0%|          | 2/460 [00:00<00:26, 17.31it/s]

...

  0%|          | 0/345 [00:00<?, ?it/s]



Imagenes Categoria: 4
1001



100%|██████████| 345/345 [00:04<00:00, 79.88it/s] 




Imagenes Categoria: 5
1002


Función ejecutada: create_training_data
Tiempo de ejecución para algoritmo: 37.68328595161438
Tamaño base de datos pickle:
6005
Función ejecutada: create_pickle_database
Tiempo de ejecución para algoritmo: 46.75252103805542


True

### Crear base de datos para Test

In [0]:
create_pickle_database(BASE_FOLDER_testFinal, Complete_cat, img_size, "TestSet", Aug_level_0)

100%|██████████| 6/6 [00:00<00:00, 76.92it/s]

Ratio data Augmentation:
1.0

Función ejecutada: create_training_data
Tiempo de ejecución para algoritmo: 0.4355928897857666
Tamaño base de datos pickle:
32
Función ejecutada: create_pickle_database
Tiempo de ejecución para algoritmo: 0.45241689682006836


True

In [0]:
create_pickle_database(BASE_FOLDER_testFinal, Complete_cat, img_size, "TestSetMax", Aug_level_max)

100%|██████████| 6/6 [00:00<00:00, 40.61it/s]


Ratio data Augmentation:
17.0

Función ejecutada: create_training_data
Tiempo de ejecución para algoritmo: 0.8210217952728271
Tamaño base de datos pickle:
544
Función ejecutada: create_pickle_database
Tiempo de ejecución para algoritmo: 1.4099750518798828


True